In [0]:
import pandas as pd
!pip install geopy
import geopy as geo 
from geopy import distance
from pandas import read_csv

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": "<application-id>",
           "fs.azure.account.oauth2.client.secret": dbutils.secrets.get(scope="<scope-name>",key="<service-credential-key-name>"),
           "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/<directory-id>/oauth2/token"}

In [0]:
dbutils.fs.mount(
  source = "abfss://<container-name>@<storage-account-name>.dfs.core.windows.net/",
  mount_point = "/mnt/data_eng_challenge",
  extra_configs = configs)

In [0]:
df_source = spark.read.format('csv').options(header='true', inferSchema='true').load('/mnt/data_eng_challenge/trips.csv')

In [0]:
df = df_source.toPandas()

In [0]:
df['origin_coord'] = df['origin_coord'].str.replace('POINT','')
df['destination_coord'] = df['destination_coord'].str.replace('POINT', '')
df['origin_coord'] = df['origin_coord'].str.replace('(','')
df['origin_coord'] = df['origin_coord'].str.replace(')','')
df['destination_coord'] = df['destination_coord'].str.replace('(','')
df['destination_coord'] = df['destination_coord'].str.replace(')','')
df['destination_coord'] = df['destination_coord'].str.lstrip()
df['origin_coord'] = df['origin_coord'].str.lstrip()
df['origin_coord'] = df['origin_coord'].str.replace(' ',' , ',1)
df['destination_coord'] = df['destination_coord'].str.replace(' ',' , ',1)
df['Distance'] = pd.NaT

for i in range(0,len(df)):
    point1 = ( df.iloc[i]['origin_coord'])
    point2 = ( df.iloc[i]['destination_coord'])
    df.loc[i]=(df.iloc[i]['region'],df.iloc[i]['origin_coord'] ,df.iloc[i]['destination_coord'],df.iloc[i]['datetime'] , df.iloc[i]['datasource'] ,distance.distance(point1 , point2).km )

In [0]:
sparkDF=spark.createDataFrame(df) 

In [0]:
jdbcHostname = "<database_server_name>.database.windows.net"
jdbcDatabase = "<database_name>"
jdbcPort = 1433
jdbcUrl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostname, jdbcPort, jdbcDatabase)
connectionProperties = {
"user" : "<database_username_or_keyvault_alias>",
"password" : "<database_password_or_keyvault_alias>",
"driver" : "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
sparkDF.write.jdbc(url=jdbcUrl, table="dbo.File3", mode = "overwrite",properties=connectionProperties)